# Import library

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
import torch.nn.functional as F
import torch.nn.init as init
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import pickle
import random
from os.path import dirname, join as pjoin
import copy
from utils.networks import *
from utils.trainer import *
from utils.formulations import *

datafolderpath = './data'
saveresultfolderpath = './result'

def report_savefig(plotname, resultfolderpath = './', formattype = 'pdf', pad_margin = 0.01):
    fig.savefig(resultfolderpath + plotname + f'.{formattype}',format=formattype, 
                bbox_inches='tight',pad_inches=pad_margin,transparent=True)
    


# Evaluation cost of PI with point forecast

## Import data

In [2]:
dict_path = os.path.join(datafolderpath, 'data_central_train_nonan.pkl')
with open(dict_path, 'rb') as pickle_file:
    data = pickle.load(pickle_file)
print(data.keys())

X_train = data['X_train']
y_train = data['y_train']
X_val = data['X_val']
y_val = data['y_val']
X_test = data['X_test']
y_test = data['y_test']

target_col = data['target_col']
features_list = data['features_list']
future_regressor = data['future_regressor']

df_train_date = data['df_train_date']
df_val_date = data['df_val_date']
df_test_date = data['df_test_date']


num_step_ahead = data['num_step_ahead']
resolution = data['resolution']

df_test_features = data['df_test_nonan'] # For feature plotting

dict_keys(['X_train', 'y_train', 'X_val', 'y_val', 'X_test', 'y_test', 'target_col', 'features_list', 'future_regressor', 'df_train_nonan', 'df_val_nonan', 'df_test_nonan', 'df_train_date', 'df_val_date', 'df_test_date', 'timerange', 'num_step_ahead', 'resolution'])


## Import result

In [4]:
allresult_filename = ['sumk_solarcentral_4step_varylambda.pkl', 'sumk_lstm45_solarcentral_4step.pkl'
                      , 'qd_solarcentral_4step.pkl'
                      , 'cwcshri_solarcentral_4step.pkl', 'qr_solarcentral_4step.pkl']

index_path = next((i for i, s in enumerate(allresult_filename) if 'sumk_' in s), None)
dict_path = os.path.join(saveresultfolderpath, allresult_filename[index_path])
with open(dict_path, 'rb') as pickle_file:
    result_sumk_ann = pickle.load(pickle_file)
print(result_sumk_ann.keys())

outputs_train_sumk = torch.from_numpy(result_sumk_ann['outputs_train'][:,:,9])
outputs_val_sumk = torch.from_numpy(result_sumk_ann['outputs_val'][:,:,9])
outputs_test_sumk = torch.from_numpy(result_sumk_ann['outputs_test'][:,:,9])


index_path = next((i for i, s in enumerate(allresult_filename) if 'qd' in s), None)
dict_path = os.path.join(saveresultfolderpath, allresult_filename[index_path])
with open(dict_path, 'rb') as pickle_file:
    result_qd = pickle.load(pickle_file)
print(result_qd.keys()) 

outputs_train_qd = result_qd['outputs_train']
outputs_val_qd = result_qd['outputs_val']
outputs_test_qd = result_qd['outputs_test']


index_path = next((i for i, s in enumerate(allresult_filename) if 'cwcshri' in s), None)
dict_path = os.path.join(saveresultfolderpath, allresult_filename[index_path])
with open(dict_path, 'rb') as pickle_file:
    result_cwcshri = pickle.load(pickle_file)
print(result_cwcshri.keys())

outputs_train_cwc = result_cwcshri['outputs_train']
outputs_val_cwc = result_cwcshri['outputs_val']
outputs_test_cwc = result_cwcshri['outputs_test']


index_path = next((i for i, s in enumerate(allresult_filename) if 'qr' in s), None)
dict_path = os.path.join(saveresultfolderpath, allresult_filename[index_path])
with open(dict_path, 'rb') as pickle_file:
    result_qr = pickle.load(pickle_file)
print(result_qr.keys())

outputs_train_qr = result_qr['outputs_train']
outputs_val_qr = result_qr['outputs_val']
outputs_test_qr = result_qr['outputs_test']

dict_path = os.path.join(saveresultfolderpath, 'point_solarcentral_4step.pkl')
with open(dict_path, 'rb') as pickle_file:
    result_point = pickle.load(pickle_file)
print(result_qr.keys())

outputs_train_point = result_point['outputs_train_point']
outputs_val_point = result_point['outputs_val_point']
outputs_test_point = result_point['outputs_test_point']


## Define model and trainer
train = trainer_multistep(num_epochs = 2000, batch_size = 2000, patience = 200) #Set the trainer

outputs_all_qd = outputs_test_qd
outputs_all_qr = outputs_test_qr
outputs_all_cwc = outputs_test_cwc
outputs_all_sumk = outputs_test_sumk
outputs_point_all = outputs_test_point
y_all = y_test
df_all_date = df_test_date

# Exclude QR in comparison
outputs_pi_all = torch.stack([outputs_all_qd, outputs_all_cwc, outputs_all_sumk], dim = 2)

if len(outputs_point_all.shape) == 2:
    print('Use only one forecast model')
    outputs_point_all = outputs_point_all.unsqueeze(-1).repeat(1, 1, outputs_pi_all.shape[2])
else:
    print('Use different forecast model')
    pass

dict_keys(['outputs_train', 'outputs_val', 'outputs_test', 'PICP_val', 'PICP_test', 'PINAW_test', 'PINALW_test', 'gamma', 'lambda'])
dict_keys(['outputs_train', 'outputs_val', 'outputs_test', 'gamma'])
dict_keys(['outputs_train', 'outputs_val', 'outputs_test', 'gamma'])
dict_keys(['outputs_train', 'outputs_val', 'outputs_test'])
dict_keys(['outputs_train', 'outputs_val', 'outputs_test'])
Use only one forecast model


## QR have crossing quantile between upper and lower bound -> exclude from comparison

In [5]:
print('  QR      1   2     3    4 step-ahead')
print(torch.sum(outputs_all_qr[:, 1::2] < outputs_all_qr[:, 0::2], axis = 0))
print('----- QR have crossing quantile, so we exclude this from comparison -----')
print('  QD    1  2  3  4 step-ahead')
print(torch.sum(outputs_all_qd[:, 1::2] < outputs_all_qd[:, 0::2], axis = 0))
print('  CWC   1  2  3  4 step-ahead')
print(torch.sum(outputs_all_cwc[:, 1::2] < outputs_all_cwc[:, 0::2], axis = 0))
print('  Sum-k 1  2  3  4 step-ahead')
print(torch.sum(outputs_all_sumk[:, 1::2] < outputs_all_sumk[:, 0::2], axis = 0))

  QR      1   2     3    4 step-ahead
tensor([0, 9, 0, 0])
----- QR have crossing quantile, so we exclude this from comparison -----
  QD    1  2  3  4 step-ahead
tensor([0, 0, 0, 0])
  CWC   1  2  3  4 step-ahead
tensor([0, 0, 0, 0])
  Sum-k 1  2  3  4 step-ahead
tensor([0, 0, 0, 0])


## Adjust point forecast to lie in PI in every methods: QD, CWC, Sum-k

In [6]:
# ADJUST point forecast based on all PI
upper_all = outputs_pi_all[:, 1::2,:]
lower_all = outputs_pi_all[:, 0::2,:]

lowest_upper = torch.min(upper_all, dim=2).values
highest_lower = torch.max(lower_all, dim=2).values
midpoint_pi = (lowest_upper + highest_lower)/2

mask_outside_pi = (outputs_point_all < lower_all) | (outputs_point_all > upper_all)
mask_outside_allmethods = torch.any(mask_outside_pi, axis=2)

point_adjusted = torch.where(mask_outside_allmethods, midpoint_pi, outputs_point_all[:,:,0])
outputs_point_all = point_adjusted.unsqueeze(-1).repeat(1, 1, outputs_pi_all.shape[2])

# Check point forecast lie in PI for all methods

upper_all_check = outputs_pi_all[:, 1::2, :]
lower_all_check = outputs_pi_all[:, 0::2, :]

print('Check that point forecast lie within the PI for all methods.')
print('        QD CW Sum-k')
print(torch.sum((outputs_point_all > upper_all_check) | (outputs_point_all < lower_all_check), axis = 0))

Check that point forecast lie within the PI for all methods.
        QD CW Sum-k
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


## Evaluate the result

### In case of all sites

In [7]:
df_site = df_all_date
pi_site = outputs_pi_all.numpy()
y_site = y_all.numpy()
y_site = np.repeat(y_site[:, :, np.newaxis], pi_site.shape[2], axis = 2)
y_pred_site = outputs_point_all.numpy() # Shape (N, num_step_ahead, num_methods)

### Cost evaluation

In [8]:
def cost_evaluation_calculation(y_true, y_pred, upper_pred, lower_pred, df_skycond, return_reserve = True, 
                   ur_pro = (5.5, 7.5), dr_pro = (0.08, 0.1), ur_def = (55, 75), dr_def = (0.8, 1)
                    , cutpoint_energy = 30, desired_capacity = 100):
    # Desired_capacity is in the unit of MW
    solar_efficiency = 0.2129
    # Conversion factor to convert irradiance to solar power W/sqm to MW
    # Jinko Solar Panel model no. 72HC 550 W: panel dimension 2278 x 1134 x 30 mm with Pmax = 550 W
    conversion_factor =((2.278*1.134/550)*desired_capacity*solar_efficiency)/4
        
    # Calculate reserve in the unit of MWh
    r_ur = (y_pred - lower_pred)*conversion_factor # The UR deployments
    r_dr = (upper_pred - y_pred)*conversion_factor # The DR deployments
    r_ur_def = np.maximum(lower_pred - y_true, 0.0)*conversion_factor # The UR deficits
    r_dr_def = np.maximum(y_true - upper_pred, 0.0)*conversion_factor # The DR deficits
    
    cutpoint_energy_ur = cutpoint_energy
    cutpoint_energy_dr = cutpoint_energy
    
    cost_ur = (r_ur <= cutpoint_energy_ur)*ur_pro[0]*r_ur + (r_ur > cutpoint_energy_ur)*ur_pro[1]*r_ur
    cost_dr = (r_dr <= cutpoint_energy_dr)*dr_pro[0]*r_dr + (r_dr > cutpoint_energy_dr)*dr_pro[1]*r_dr

    cutpoint_energy_ur_def = cutpoint_energy
    cutpoint_energy_dr_def = cutpoint_energy
        
    cost_ur_def = (r_ur_def <= cutpoint_energy_ur_def)*ur_def[0]*r_ur_def + (r_ur_def > cutpoint_energy_ur_def)*ur_def[1]*r_ur_def
    cost_dr_def = (r_dr_def <= cutpoint_energy_dr_def)*dr_def[0]*r_dr_def + (r_dr_def > cutpoint_energy_dr_def)*dr_def[1]*r_dr_def
    
    # (N, (cost_ur, cost_dr, cost_ur_def, cost_dr_def), num_step_ahead, num_methods)
    cost_array = np.stack([cost_ur, cost_ur_def, cost_dr, cost_dr_def], axis = 1)
    cost_array_list = []
    sky_conditions = ['clearsky', 'partlycloudy', 'cloudy']

    for i, condition in enumerate(sky_conditions):
        sky_condition_mask = (df_skycond['skycondition'] == condition).values[:, np.newaxis, np.newaxis, np.newaxis]
        cost_array_skycond = sky_condition_mask*cost_array
        cost_array_list.append(cost_array_skycond)

    # (N, (cost_ur, cost_dr, cost_ur_def, cost_dr_def), num_step_ahead, (clearsky, partlycloudy, cloudy), num_methods)
    # Divide by four to convert in unit of $/MWh
    cost_array_total = np.stack(cost_array_list, axis = 3)
    
    if return_reserve:
        return cost_array_total, np.stack((r_ur, r_ur_def, r_dr, r_dr_def), axis = 3)
    else:
        return cost_array_total

In [9]:
upper_pred_site = pi_site[:, 1::2, :] # Shape (N, num_step_ahead, num_methods)
lower_pred_site = pi_site[:, 0::2, :]# Shape (N, num_step_ahead, num_methods)

cost_array_total_10x, reserve = cost_evaluation_calculation(y_site, y_pred_site, upper_pred_site, lower_pred_site, df_site,  
                   ur_pro = (5.5, 1.5*5.5), dr_pro = (0.08, 1.5*0.08), ur_def = (50, 50), dr_def = (30, 30)
                                                            , cutpoint_energy = 10, desired_capacity = 100)

cost_array_total_100x, reserve = cost_evaluation_calculation(y_site, y_pred_site, upper_pred_site, lower_pred_site, df_site,  
                   ur_pro = (5.5, 1.5*5.5), dr_pro = (0.08, 1.5*0.08), ur_def = (500, 500), dr_def = (30, 30)
                                                             , cutpoint_energy = 10, desired_capacity = 100)

r_ur = reserve[:, :, :, 0]
r_ur_def = reserve[:, :, :, 1]
r_dr = reserve[:, :, : ,2]
r_dr_def = reserve[:, :, :, 3]

print('------------------------ PICP --------------------------')
print('    QD    CWC   Sum-k    ')
print(np.sum((y_site <= upper_pred_site) & (y_site >= lower_pred_site), axis = 0)/y_site.shape[0])
print('--------------------------------------------------------')

# (N, num_cost, num_step_ahead, (clearsky, partlycloudy, cloudy), num_methods)
step_ahead = 1
print('----- R_ur -----')
print(np.sum(r_ur[:, step_ahead - 1], axis = 0))
print('----- R_ur_def -----')
print(np.sum(r_ur_def[:, step_ahead - 1], axis = 0))
print('----- R_dr -----')
print(np.sum(r_dr[:, step_ahead - 1], axis = 0))
print('----- R_dr_def -----')
print(np.sum(r_dr_def[:, step_ahead - 1], axis = 0))

print(f'---------- Cost step {step_ahead}-ahead $(x10^6) ----------')
print(f'==================== 10X ====================')
result = np.sum(cost_array_total_10x, axis = (0, 3))/1000000
np.set_printoptions(precision=3, suppress=True)  # 3 decimals, no scientific notation
print('     QD        CWC      SUM-k')
print(result[:,step_ahead-1])

print(f'---------- Total cost {step_ahead} step $(x10^6) ----------')
result = np.sum(cost_array_total_10x, axis = (0, 1, 3))/1000000
print(result[step_ahead-1, :])

print(f'==================== 100X ====================')
result = np.sum(cost_array_total_100x, axis = (0, 3))/1000000
np.set_printoptions(precision=3, suppress=True)  # 3 decimals, no scientific notation
print('     QD        CWC      SUM-k')
print(result[:,step_ahead-1])

print(f'---------- Total cost {step_ahead} step $(x10^6) ----------')
result = np.sum(cost_array_total_100x, axis = (0, 1, 3))/1000000
print(result[step_ahead-1, :])

# (N, num_cost, num_step_ahead, (clearsky, partlycloudy, cloudy), num_methods)
step_ahead = 4
print('----- R_ur -----')
print(np.sum(r_ur[:, step_ahead - 1], axis = 0))
print('----- R_ur_def -----')
print(np.sum(r_ur_def[:, step_ahead - 1], axis = 0))
print('----- R_dr -----')
print(np.sum(r_dr[:, step_ahead - 1], axis = 0))
print('----- R_dr_def -----')
print(np.sum(r_dr_def[:, step_ahead - 1], axis = 0))

print(f'---------- Cost step {step_ahead}-ahead $(x10^6) ----------')
print(f'==================== 10X ====================')
result = np.sum(cost_array_total_10x, axis = (0, 3))/1000000
np.set_printoptions(precision=3, suppress=True)  # 3 decimals, no scientific notation
print('     QD        CWC      SUM-k')
print(result[:,step_ahead-1])

print(f'---------- Total cost {step_ahead} step $(x10^6) ----------')
result = np.sum(cost_array_total_10x, axis = (0, 1, 3))/1000000
print(result[step_ahead-1, :])

print(f'==================== 100X ====================')
result = np.sum(cost_array_total_100x, axis = (0, 3))/1000000
np.set_printoptions(precision=3, suppress=True)  # 3 decimals, no scientific notation
print('     QD        CWC      SUM-k')
print(result[:,step_ahead-1])

print(f'---------- Total cost {step_ahead} step $(x10^6) ----------')
result = np.sum(cost_array_total_100x, axis = (0, 1, 3))/1000000
print(result[step_ahead-1, :])

------------------------ PICP --------------------------
    QD    CWC   Sum-k    
[[0.895152   0.89541789 0.89204999]
 [0.90171054 0.90153328 0.89284765]
 [0.89267039 0.90046973 0.89435434]
 [0.88026234 0.90100151 0.89550651]]
--------------------------------------------------------
----- R_ur -----
[50485.19456807 46504.20605603 44832.8780441 ]
----- R_ur_def -----
[1301.98143333 1863.15137397 2262.90804955]
----- R_dr -----
[36475.34535946 39787.23089453 39560.466128  ]
----- R_dr_def -----
[1564.83699428 1524.86240251 1793.59392388]
---------- Cost step 1-ahead $(x10^6) ----------
==================== 10X ====================
     QD        CWC      SUM-k
[[0.303 0.274 0.251]
 [0.065 0.093 0.113]
 [0.003 0.003 0.003]
 [0.047 0.046 0.054]]
---------- Total cost 1 step $(x10^6) ----------
[0.418 0.416 0.421]
==================== 100X ====================
     QD        CWC      SUM-k
[[0.303 0.274 0.251]
 [0.651 0.932 1.131]
 [0.003 0.003 0.003]
 [0.047 0.046 0.054]]
---------- Total

## Plotting group bar chart

In [ ]:
# Get the cost array shapes (assuming already loaded)
step_ahead = 4
cost_array_total_allcase = np.stack([cost_array_total_10x, cost_array_total_100x], axis = 5)
cost_evaluation = cost_array_total_allcase[:, :, step_ahead - 1, :, :, :]

formname = ['QD', r'$\mathrm{CWC}_{\mathrm{Shri}}$', r'Sum-$k$']
costname = ['UR provision', 'UR deficit', 'DR provision', 'DR deficit']
groupname = ['50', '500']
colors = plt.cm.Paired(np.linspace(0, 1, len(costname)))
colors[2] = [0.0, 0.98039216, 0.60392157, 1.0]

# === Grouping Setup ===
n_methods = len(formname)
n_costs = len(costname)
n_group = len(groupname)

# Axis based on dimensionality
fig, ax = plt.subplots(ncols = n_group, figsize=(10, 5))

# Reduce dimensions to 3D: shape = (n_costs, n_methods, 2 cases (10x, 100x))
data = np.sum(cost_evaluation, axis = (0, 2)) / 1_000_000

# Formatting
for k in range(n_group):
    # Plot stacked bars for 10x group
    x = np.arange(n_methods)*0.4
    for i in range(n_costs):
        bottom = np.sum(data[:i, :, k], axis=0)
        ax[k].bar(x, data[i,:,k], bottom=bottom, width=0.2, color=colors[i], linewidth=1,
               label=costname[i])
    
    ymax = np.max(np.sum(data, axis=0), axis = 0)[k]
    # Add total value on top of each stacked bar
    bar_height = np.sum(data[:,:,k], axis = 0)
    for xi, data_i in zip(x, bar_height):
        ax[k].text(
            xi,
            data_i + ymax * 0.005,  # safe offset
            f'{data_i:.2f}',
            ha='center', va='bottom',
            fontsize=11
        )
    
    # X-axis labels
    xticks = x       
    ax[k].set_xticks(xticks)
    ax[k].set_xticklabels(formname, fontsize=12)

    # Labels, title, legend
    ax[k].set_ylim(0, ymax * 1.2) 
    ax[k].set_ylabel(r'Operational cost (\$ $10^{6}$)', fontsize=14)
    ax[k].set_title(f'The value of lost load = ${groupname[k]}/MWh', fontsize=14)

    # Aesthetics
    ax[k].grid(True, axis='y', linestyle='--', alpha=0.5)
    for spine in ax[k].spines.values():
        spine.set_linewidth(0.4)
        
# Global legend after all subplots are done
handles, labels = ax[0].get_legend_handles_labels()
# fig.suptitle(f'{step_ahead}-step ahead reserve cost', fontsize = 15)
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0), ncol=4, fontsize=12, frameon=False)
    
plt.tight_layout()
plt.show()